<!--TITLE: Exercise Convolution and ReLU-->
# Introduction #

In these exercises, you'll explore the feature extraction operations on your own. You'll also see how you can open up a convnet to look at the feature maps deep within its layers.

Run the cell below to get started!

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.computer_vision.ex2 import *

# Apply Transformations

Run the following cell to load an image we'll use for the next few exercises.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

#TODO: let user select own image (from url maybe?)
image_path = './images/car_feature.jpg'
image = np.array(Image.open(image_path)

plt.figure(figsize=(6, 6))
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.show();

### 1) Define Kernel

First define a kernel. You have your choice of what kind of kernel to apply. There are a couple cells below that can give you some ideas. Once you've found one you like, just type in values here:

In [ ]:
# YOUR CODE HERE: Define this just like you would a numpy array.
kernel = tf.constant(
    ____
)

q_1.check()

In [ ]:
kernel = tf.constant([
    [-2, -1, 0],
    [-1, 1, 1],
    [0, 1, 2],
])
#%%RM_IF(PROD)%%
q_1.assert_check_passed()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_1.hint()
#_COMMENT_IF(PROD)_
q_1.solution()

Run this cell to see some standard kernels used in image processing.

In [ ]:
from visiontools import edge, bottom_sobel, emboss_se, sharpen

kernels = [edge, bottom_sobel, emboss, sharpen]
names = ["Edge Detect", "Bottom Sobel", "Emboss", "Sharpen"]

plt.figure(figsize=(12, 12))
for i, (kernel, name) in enumerate(zip(kernels, names)):
    plt.subplot(1, 4, i+1)
    visiontools.show_kernel(kernel)
    plt.title(name)
plt.tight_layout()

Alternatively, you can use this cell to see a random kernel from the first layer of the VGG16 model we saw in Lesson 1.

In [ ]:
layer = vgg.get_layer('block1_conv1')
weights = layer.get_weights()[0]

# choose an input channel: 0 for blue, 1 for green, 2 for red
input_index = np.random.randint(0, weights.shape[2])

# choose a filter 0-63
filter_index = np.random.randint(0, weights.shape[3])


# get weights and normalize
krn = weights[:, :, input_index, filter_index]
krn -= krn.mean()
krn /= (krn.std() + 1e-5)
krn = krn.round(decimals=1)

print("Input Channel: {}  Filter: {}".format(input_index, filter_index))
visiontools.show_kernel(krn)

### 2) Apply Convolution

Now filter the image with your kernel using the convolution function in `tf.nn`.

In [ ]:
# YOUR CODE HERE: Apply convolution to image
image_filter = ____

q_2.check()

In [ ]:
#%%RM_IF(PROD)%%
image_filter = tf.nn.conv2d(
    input=image,
    filters=kernel,
    strides=1,
    padding='SAME',
)
q_2.assert_check_passed()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_2.hint()
#_COMMENT_IF(PROD)_
q_2.solution()

Run the next cell to see the result!

In [ ]:
plt.imshow(
    # Reformat for plotting
    tf.squeeze(image_filter)
)
plt.axis('off')
plt.show();

### 3) Apply ReLU

Now detect the feature with `tf.nn.relu`.

In [ ]:
# YOUR CODE HERE
image_detect = ___

q_3.check()

In [ ]:
#%%RM_IF(PROD)%%
image_detect = tf.nn.relu(image_filter)
q_3.assert_check_passed()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_3.hint()
#_COMMENT_IF(PROD)_
q_3.solution()

Run the next cell to see the effect of the ReLU!

In [ ]:
plt.imshow(
    # Reformat for plotting
    tf.squeeze(image_detect)
)
plt.axis('off')
plt.show();

# Conclusion #

That's all for this exercise!

Here is a fun website where you can interactively explore the image kernels: [Image Kernels Explained Visually](https://setosa.io/ev/image-kernels/).
